# first exploring the data

In [1]:
# Import packages
#import plotly.express as px
import pandas as pd
#from statsmodels.tsa.stattools import adfuller
import numpy as np
import sklearn as sk
## import pmdarima
import tensorflow
## import fprophet
## from pmdarima.arima import auto_arima
## from fbprophet import Prophet
## from tensorflow.keras.models import Sequential
## from tensorflow.keras.layers import LSTM, Dense
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


C:\Users\alaaw\AppData\Local\Temp\ipykernel_26868\631163819.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Read in the data
df = pd.read_excel("Sample Data Set (1).xlsx")
df.head()

,Reporting Economy,Product/Sector,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,SI3_AGG - TO - Total merchandise,10510291.0,12131279.0,14031816.0,16168718.0,12564776.0,15301659.0,18337603.0,18508981.0,18958769.0,18999593.0,16554643.0,16036335.0,17741325.0,19550037.0,19017108.0,17653275.0,22365766.0,24925766.0
1,World,SI3_AGG - AG - Agricultural products,847715.0,942450.0,1131691.0,1342823.0,1177136.0,1354429.0,1649389.0,1655099.0,1731531.0,1746687.0,1565686.0,1585693.0,1731381.0,1810271.0,1788200.0,1809475.0,2148665.0,2325521.0
2,World,SI3_AGG - AGFO - Food,680936.0,753836.0,915653.0,1114698.0,998004.0,1115790.0,1350316.0,1369778.0,1445339.0,1480252.0,1332886.0,1357202.0,1473948.0,1533561.0,1533159.0,1573538.0,1836335.0,2007666.0
3,World,SI3_AGG - MI - Fuels and mining products,1810185.0,2319402.0,2674500.0,3554158.0,2299789.0,3074398.0,4122493.0,4164125.0,4051300.0,3720251.0,2380017.0,2033684.0,2611396.0,3264056.0,3102597.0,2326889.0,3670710.0,4950975.0
4,World,SI3_AGG - MIFU - Fuels,1457315.0,1801129.0,2047440.0,2884170.0,1838528.0,2395065.0,3268505.0,3399965.0,3309479.0,3026072.0,1808811.0,1492255.0,1948478.0,2528513.0,2370333.0,1571511.0,2555970.0,3865877.0


checking the number of mising values in the data for each each column in our dataset

In [ ]:
print(df.isnull().sum())

In [ ]:
## slicing the desciptive data off the dataframe
sliced_df = df.iloc[:, 2:]

Exploring futher, to find out wither some rows should be deleted or not

In [ ]:
missing_values_per_row = sliced_df.isnull().sum(axis=1)
rows_with_missing_values = missing_values_per_row[missing_values_per_row > 0]
sorted_indices = missing_values_per_row.sort_values(ascending=False).index
top_10_indices = sorted_indices[:10]
missing_values_df = missing_values_per_row[top_10_indices].reset_index()
missing_values_df.columns = ['index', 'missing_values']
# Merge the original DataFrame with the missing values count
result_df = df.loc[top_10_indices].reset_index().merge(missing_values_df, on='index')

print(result_df)

In [ ]:
rows_with_over_15_missing_values = missing_values_per_row[missing_values_per_row > 15]

# Count the number of such rows
num_rows_with_over_15_missing = rows_with_over_15_missing_values.shape[0]

# Step 6: Calculate the percentage of these rows over the overall data
total_rows = df.shape[0]
percentage = (num_rows_with_over_15_missing / total_rows) * 100

# Output the results
print(f"Number of rows with more than 15 missing values: {num_rows_with_over_15_missing}")
print(f"Percentage of rows with more than 15 missing values: {percentage:.2f}%")

it seems like deleting the rows with missing values higer then 15, would mean sacrificing 1.87%. so we will explore some other options before choosing this option

Afganistan has alot of missing data so i'm going to test the preprocessing on that. the data seems to be inconcsistant from region to region.

In [4]:
# Filter the data for Afghanistan
afghanistan_data = df[df['Reporting Economy'] == 'Afghanistan']

# Print the data for Afghanistan
print(afghanistan_data)

   Reporting Economy                                     Product/Sector  \
18       Afghanistan                   SI3_AGG - TO - Total merchandise   
19       Afghanistan               SI3_AGG - AG - Agricultural products   
20       Afghanistan                              SI3_AGG - AGFO - Food   
21       Afghanistan           SI3_AGG - MI - Fuels and mining products   
22       Afghanistan                             SI3_AGG - MIFU - Fuels   
23       Afghanistan                        SI3_AGG - MA - Manufactures   
24       Afghanistan                    SI3_AGG - MAIS - Iron and steel   
25       Afghanistan                         SI3_AGG - MACH - Chemicals   
26       Afghanistan                 SI3_AGG - MACHPH - Pharmaceuticals   
27       Afghanistan  SI3_AGG - MAMT - Machinery and transport equip...   
28       Afghanistan    SI3_AGG - MAMTOF - Office and telecom equipment   
29       Afghanistan  SI3_AGG - MAMTOTEP - Electronic data processin...   
30       Afghanistan  SI3

Tuvalu seems to have a lot of missing data as well so i'll explore that too

In [3]:
# Filter the data for Afghanistan
tuvalu_data = df[df['Reporting Economy'] == 'Tuvalu']

# Print the data for Afghanistan
print(tuvalu_data)

     Reporting Economy                                     Product/Sector  \
3276            Tuvalu                   SI3_AGG - TO - Total merchandise   
3277            Tuvalu           SI3_AGG - MI - Fuels and mining products   
3278            Tuvalu                             SI3_AGG - MIFU - Fuels   
3279            Tuvalu                        SI3_AGG - MA - Manufactures   
3280            Tuvalu  SI3_AGG - MAMT - Machinery and transport equip...   
3281            Tuvalu    SI3_AGG - MAMTOF - Office and telecom equipment   
3282            Tuvalu  SI3_AGG - MAMTOTEP - Electronic data processin...   
3283            Tuvalu  SI3_AGG - MAMTOTTL - Telecommunications equipment   
3284            Tuvalu             SI3_AGG - MAMTTE - Transport equipment   
3285            Tuvalu             SI3_AGG - MAMTAU - Automotive products   

      2005  2006  2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  \
3276   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

In [13]:
### Select only numeric columns for imputation
numeric_data = df.select_dtypes(include=['number'])


In [17]:
print(numeric_data)

            2005        2006        2007        2008        2009        2010  \
0     10510291.0  12131279.0  14031816.0  16168718.0  12564776.0  15301659.0   
1       847715.0    942450.0   1131691.0   1342823.0   1177136.0   1354429.0   
2       680936.0    753836.0    915653.0   1114698.0    998004.0   1115790.0   
3      1810185.0   2319402.0   2674500.0   3554158.0   2299789.0   3074398.0   
4      1457315.0   1801129.0   2047440.0   2884170.0   1838528.0   2395065.0   
...          ...         ...         ...         ...         ...         ...   
3520         0.0         0.0         0.0         1.0         7.0         0.0   
3521         8.0       165.0        84.0        90.0        16.0        17.0   
3522         6.0        16.0        28.0        77.0        10.0         8.0   
3523        14.0        23.0        67.0        74.0        17.0        18.0   
3524         7.0        17.0        18.0        47.0        16.0         7.0   

            2011        2012        201

since the data depends on the region(rows) as well as the time i'll use KNNimputer which uses a proximaty(distance) metric to find the missing value. choose the n neighbors useing a hyper pramater that should be tuned

In [18]:
imputer = KNNImputer(n_neighbors=5)


In [19]:
## data preprocessing
imputed_data = imputer.fit_transform(numeric_data)
imputed_df = pd.DataFrame(imputed_data, columns=numeric_data.columns)
print(imputed_df.head())

         2005        2006        2007        2008        2009        2010  \
0  10510291.0  12131279.0  14031816.0  16168718.0  12564776.0  15301659.0   
1    847715.0    942450.0   1131691.0   1342823.0   1177136.0   1354429.0   
2    680936.0    753836.0    915653.0   1114698.0    998004.0   1115790.0   
3   1810185.0   2319402.0   2674500.0   3554158.0   2299789.0   3074398.0   
4   1457315.0   1801129.0   2047440.0   2884170.0   1838528.0   2395065.0   

         2011        2012        2013        2014        2015        2016  \
0  18337603.0  18508981.0  18958769.0  18999593.0  16554643.0  16036335.0   
1   1649389.0   1655099.0   1731531.0   1746687.0   1565686.0   1585693.0   
2   1350316.0   1369778.0   1445339.0   1480252.0   1332886.0   1357202.0   
3   4122493.0   4164125.0   4051300.0   3720251.0   2380017.0   2033684.0   
4   3268505.0   3399965.0   3309479.0   3026072.0   1808811.0   1492255.0   

         2017        2018        2019        2020        2021        2022 

In [20]:
# Filter the data for Afghanistan
afghanistan_data = imputed_df[df['Reporting Economy'] == 'Afghanistan']

# Print the data for Afghanistan
print(afghanistan_data)

     2005   2006   2007   2008   2009   2010   2011   2012   2013   2014  \
18  384.0  416.0  454.0  540.0  403.0  388.0  376.0  429.0  515.0  570.0   
19   62.8  106.0  191.0  311.0  251.0  197.0  196.0   76.0   70.0   83.0   
20   59.8   93.6  151.2  284.0  221.0  156.0  147.0   75.0   70.0   83.0   
21    0.0    0.0    0.0    0.0    1.0    1.0    1.0    0.6    0.4    0.0   
22    8.6   11.4    7.4    0.0   11.8   13.6    7.8    7.6   15.2    5.8   
23  126.0   58.4  110.4  187.0   73.0   76.0   53.0   73.0   73.0   84.0   
24    3.2    1.0    0.0    0.2    0.0    0.0    0.0    0.0    0.2    0.2   
25    0.6    0.6    0.4    0.2    0.2    0.4    0.0    0.0    0.2    0.0   
26    3.2    1.0    0.0    0.2    0.0    0.0    0.0    0.0    0.2    0.2   
27    0.8    2.6    1.2    4.8    0.2    2.2    7.6    1.6    3.4    2.4   
28   11.6   12.0   11.4   34.0   15.0   10.2   17.6   18.0    7.0   63.8   
29   12.0    0.2    0.2    0.2    2.0    1.0    6.6    0.0    2.6    0.2   
30   11.6   

In [ ]:
# Filter the data for Afghanistan
tuvalu_data = imputed_df[df['Reporting Economy'] == 'Tuvalu']

# Print the data for Afghanistan
print(tuvalu_data)

In [21]:
print(imputed_df)

            2005        2006        2007        2008        2009        2010  \
0     10510291.0  12131279.0  14031816.0  16168718.0  12564776.0  15301659.0   
1       847715.0    942450.0   1131691.0   1342823.0   1177136.0   1354429.0   
2       680936.0    753836.0    915653.0   1114698.0    998004.0   1115790.0   
3      1810185.0   2319402.0   2674500.0   3554158.0   2299789.0   3074398.0   
4      1457315.0   1801129.0   2047440.0   2884170.0   1838528.0   2395065.0   
...          ...         ...         ...         ...         ...         ...   
3520         0.0         0.0         0.0         1.0         7.0         0.0   
3521         8.0       165.0        84.0        90.0        16.0        17.0   
3522         6.0        16.0        28.0        77.0        10.0         8.0   
3523        14.0        23.0        67.0        74.0        17.0        18.0   
3524         7.0        17.0        18.0        47.0        16.0         7.0   

            2011        2012        201

In [25]:
X_data=df.iloc[:,:2]
X_data.head()


,Reporting Economy,Product/Sector
0,World,SI3_AGG - TO - Total merchandise
1,World,SI3_AGG - AG - Agricultural products
2,World,SI3_AGG - AGFO - Food
3,World,SI3_AGG - MI - Fuels and mining products
4,World,SI3_AGG - MIFU - Fuels


In [28]:
final_imputed_df = pd.concat([X_data, imputed_df], axis=1)
final_imputed_df.head()

,Reporting Economy,Product/Sector,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,SI3_AGG - TO - Total merchandise,10510291.0,12131279.0,14031816.0,16168718.0,12564776.0,15301659.0,18337603.0,18508981.0,18958769.0,18999593.0,16554643.0,16036335.0,17741325.0,19550037.0,19017108.0,17653275.0,22365766.0,24925766.0
1,World,SI3_AGG - AG - Agricultural products,847715.0,942450.0,1131691.0,1342823.0,1177136.0,1354429.0,1649389.0,1655099.0,1731531.0,1746687.0,1565686.0,1585693.0,1731381.0,1810271.0,1788200.0,1809475.0,2148665.0,2325521.0
2,World,SI3_AGG - AGFO - Food,680936.0,753836.0,915653.0,1114698.0,998004.0,1115790.0,1350316.0,1369778.0,1445339.0,1480252.0,1332886.0,1357202.0,1473948.0,1533561.0,1533159.0,1573538.0,1836335.0,2007666.0
3,World,SI3_AGG - MI - Fuels and mining products,1810185.0,2319402.0,2674500.0,3554158.0,2299789.0,3074398.0,4122493.0,4164125.0,4051300.0,3720251.0,2380017.0,2033684.0,2611396.0,3264056.0,3102597.0,2326889.0,3670710.0,4950975.0
4,World,SI3_AGG - MIFU - Fuels,1457315.0,1801129.0,2047440.0,2884170.0,1838528.0,2395065.0,3268505.0,3399965.0,3309479.0,3026072.0,1808811.0,1492255.0,1948478.0,2528513.0,2370333.0,1571511.0,2555970.0,3865877.0


In [29]:
##output the data 
final_imputed_df.to_excel('final_imputed_df.xlsx', index=False)

After filling in the NaN data segregate the independent(reporting economy, product/sector) and dependent veriables (year values). to be able to use the numbers for training.

In [ ]:
##extracting the vlaues of the independent variable, all the rows and the first two columns
X = df.iloc[:,:1].values
##extracting the values of the dependent variables, only the the columns after the first two
Y = df.iloc[:, 2:].values